In [1]:
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union 
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from scipy.signal import stft 
from scipy.signal import butter
from librosa.feature import mfcc

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [6]:
#Spectograms, short-time Fourier Transform
stft_transform = FunctionTransformer(stft, kw_args={'window':'hann', 'nfft':None})

In [8]:
#bandpass filtering
band_filter = FunctionTransformer(butter, kw_args={'N':1, 'btype':['low', 'high']})

In [13]:
#Scaling, sklearn, StandardScaler
scaler = make_pipeline(StandardScaler())

In [10]:
#MFCC (librosa), params: n_coeff
mfcc_feat = FunctionTransformer(mfcc, kw_args={'n_mfcc':20})

In [11]:
mfcc_feat.get_params()

{'accept_sparse': False,
 'check_inverse': True,
 'feature_names_out': None,
 'func': <function librosa.feature.spectral.mfcc(*, y=None, sr=22050, S=None, n_mfcc=20, dct_type=2, norm='ortho', lifter=0, **kwargs)>,
 'inv_kw_args': None,
 'inverse_func': None,
 'kw_args': {'n_mfcc': 20},
 'validate': False}

In [ ]:
#Mean along the x-axis(time) (by hand) 


In [14]:
preproc = make_column_transformer(
            (scaler, make_column_selector(dtype_include=['float64'])),
            remainder='passthrough')

In [15]:
pipe = make_pipeline(stft_transform, 
                     band_filter, 
                     preproc, 
                     mfcc_feat)

In [18]:
pipe.get_params()

{'memory': None,
 'steps': [('functiontransformer-1',
   FunctionTransformer(func=<function stft at 0x13f99a1f0>,
                       kw_args={'nfft': None, 'window': 'hann'})),
  ('functiontransformer-2',
   FunctionTransformer(func=<function butter at 0x13f8f6e50>,
                       kw_args={'N': 1, 'btype': ['low', 'high']})),
  ('columntransformer',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('pipeline',
                                    Pipeline(steps=[('standardscaler',
                                                     StandardScaler())]),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x14dfc41f0>)])),
  ('functiontransformer-3',
   FunctionTransformer(func=<function mfcc at 0x141ee6160>, kw_args={'n_mfcc': 20}))],
 'verbose': False,
 'functiontransformer-1': FunctionTransformer(func=<function stft at 0x13f99a1f0>,
                     kw_args={'nfft': None, 'window': 'han

In [ ]:
#params dictionary, keyword = name of the steps in pipeline, values = params
params = {'stft_transform': 'functiontransformer-1__kw_args', 
         'band_filter': 'functiontransformer-2__kw_args', 
         'mfcc_feat': 'functiontransformer-3__kw_args', 
         }